In [101]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import cufflinks as cf
import glob


cf.go_offline()

sns.set(style="darkgrid")

In [102]:
data_paths = glob.glob('../PostHog/data/*.csv')
frames = [pd.read_csv(path, names=['title', 'language', 'icon', 'id', 'timestamp']) for path in data_paths]
data = pd.concat(frames)
data = data.drop_duplicates(subset=['id'])
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['title'] = data['title'].str.strip()
data.head()
data.groupby(pd.Grouper(key='timestamp', axis=0, freq='1D'))['id'].count().iplot(kind='bar')

In [103]:
df = data.groupby(by=["icon", "title", "language"])[['id']].nunique()
print(len(df))
filtered = df[df['id'] >= 3]
print(len(filtered))
filtered = filtered.reset_index()
filtered.sort_values(by='id', ascending=False).head()

1398531
276691


,icon,title,language,id
2340,alarm.fill,Wake up!,en,186074
182722,moon.fill,Sleep Well!,en,83844
973,alarm.fill,Aufwachen!,de,45220
84096,custom.shower,Shower,en,34537
198384,noun.meal,Lunch,en,20600


In [104]:
df.reset_index().groupby(by=["id"])[["id"]].count().iplot(kind='bar')

In [105]:
filtered.groupby(by=["language"])['id'].count().sort_values(ascending=False).iplot(kind='bar')

In [106]:
import json

for language in filtered["language"].drop_duplicates():
    export = filtered[filtered["language"] == language][['icon', 'title']]
    export.rename(columns = {'icon': 'label', 'title': 'text'}, inplace=True)
    export.to_csv(f'data/{language}.csv', index=False)
    print(language, len(export))
export = filtered[['icon', 'title']]
export.rename(columns = {'icon': 'label', 'title': 'text'}, inplace=True)
export.to_csv(f'data/all.csv', index=False)


en 155195
fr 11866
de 35515
es 9348
it 7396
pt 4269
nb 1391
ro 326
nl 10999
zh 12195
th 4961
sv 2235
pl 5985
vi 1870
fi 530
ru 3831
ar 989
ja 4800
tr 614
ca 234
id 424
hr 168
uk 345
da 1059
hu 50
fa 30
ko 64
ur 2


In [107]:
export = filtered
export.rename(columns={'icon': 'label', 'title': 'text'}, inplace=True)
export = export[['label', 'text']]
export.to_json('user_icons_learn.json', orient="records")
